In [ ]:
pip install langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
pip install transformers

In [ ]:
pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 58.0 MB/s eta 0:00:00


In [ ]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install python-decouple

In [ ]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.3 MB/s eta 0:00:00


In [ ]:
pip install langchain-groq

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import re
from langchain.text_splitter  import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import re

def text_from_website(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = bs(response.content, 'html.parser')

        for unwanted in soup(["script", "style", "header", "footer", "nav", "aside"]):
            unwanted.extract()

        content = []
        for tag in soup.find_all(['p', 'h1', 'h2']):
            content.append(tag.get_text())

        combined_text = ' '.join(content).strip()

        cleaned_text = re.sub(r"(\w)-\n(\w)", r"\1\2", combined_text)
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

        return cleaned_text
    except Exception as e:
        return f"Error: {e}"


In [ ]:
website_text=text_from_website('https://botpenguin.com/')

In [ ]:
website_text

'Engage, Converse and Convert your visitors using AI Chatbot Agent BotPenguin’s Chatbot Maker & Live Chat for Website, WhatsApp, Facebook, Instagram & Telegram Generate 10x more leads Solve up to 80% of customer queries Engage 70% more visitors Earn 90% more revenue Performance Highlights 193+ Countries Served 100Mn+ Messages Consumed 40k+ Total Customers 120+ Partners An AI chatbot that can be anywhere, anytime Keep your customers happy and engaged with an omnichannel presence across all popular platforms with BotPenguin’s chatbot maker platform Market and sell products on WhatsApp. Utilize AI for personalized conversation and grow your business with BotPenguin’s chatbot maker. Serve your customers on Facebook, 24x7. Enhance user experience with the best customer support ever. Run our Telegram chatbot to create your own custom chatbot tailored to your interests. Sell products directly through Instagram and automate replies to messages, stories, and comments to enhance user experience.

In [ ]:
def chunk_text(text):
  '''splitting the corpus into small chunks because LLM have limited context window.
    Splitting text into chunks ensures each chunk fits within this window for better understanding and processing.
    Here we are using RecursiveCharacterTextSplitter from langchain
    '''
  text_splitter = RecursiveCharacterTextSplitter(
            separators=['\n\n', '\n', '.', ','],
            chunk_size=1000,
            chunk_overlap=200,
            )
  texts = text_splitter.split_text(text)
  return texts

texts=chunk_text(website_text)

In [ ]:
def embedding(texts):
  '''
    Then perform the vectorization on those chunks and convert into embedding and stored in vectorstore
    here im using huggingfaceebmbedding with the model which i used to perform retrieve data
    and used faiss vectorstore to store the vectors
    FAISS demonstrates exceptional proficiency in handling high-dimensional data with remarkable speed and efficiency.
    '''

  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
  vector_store = FAISS.from_texts(texts, embeddings)
  return vector_store

vector_store=embedding(texts)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

··········


In [ ]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well.
"""

retriever=vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x788cadcf9ff0>, search_kwargs={})

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    """
You are an assistant for question-answering tasks.
answer only with in 3 sentence
Use the provided context only to answer the following question:

<context>
{context}
</context>

Question: {input}
"""
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "what is BotPenguin"})
response["answer"]

'BotPenguin is an AI chatbot platform that provides native integrations with over 80 tools and applications, allowing for seamless data exchange and automation.'

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory( k=5)

In [ ]:
while True:
  query = input("Ask a question about the website ('END' to exit): ")
  if query.upper() == "END":  # Check for "END/end"
      break
  response = rag_chain.invoke({"input": "what is BotPenguin"})
  answer=response["answer"]



  memory.save_context({"input": query}, {"output": answer})
  print('---'*10)
  print(answer)

Ask a question about the website ('END' to exit): end
